In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import pyGadget
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc('text', usetex=True)

In [ ]:
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

# Sink Accretion History

In [ ]:
simname = {'vanilla':r'J = 0',
        'X0': '',#r'J = J$_0$', 
        'X1': '',#r'J = 10 J$_0$', 
        'X2': '',#r'J = 100 J$_0$', 
        'J0': r'J = J$_0$', 
        'J1': r'J = 10 J$_0$', 
        'J2': r'J = 100 J$_0$', 
        'J3': r'J = 1000 J$_0$'}
name = ['vanilla', 'J0', 'J1', 'J2', 'J3', 'X0', 'X1', 'X2']

In [ ]:
sinkdata = pd.HDFStore(os.getenv("HOME")+'/data/stampede/sinkdata.hdf5')
data0 = sinkdata['vanilla']
data1 = sinkdata['XR_sfr_1e-3']
data2 = sinkdata['XR_sfr_1e-2']
data3 = sinkdata['XR_sfr_1e-1']
data4 = sinkdata['xr_tau2_J0']
data5 = sinkdata['xr_tau2_J1']
data6 = sinkdata['xr_tau2_J2']
sinkdata.close()

In [ ]:
sinks0 = data0.set_index('time')
sinks1 = data1.set_index('time')
sinks2 = data2.set_index('time')
sinks3 = data3.set_index('time')
sinks4 = data4.set_index('time')
sinks5 = data5.set_index('time')
sinks6 = data6.set_index('time')

In [ ]:
palette = ["#4C72B0", "#55A868", "#C44E52", "#8172B2", "#CCB974", "#64B5CD", "#8172B2"]
sns.palplot(palette)

In [ ]:
with sns.color_palette(palette, n_colors=7):
    fig, [[ax0,ax1], [ax2,ax3]] = plt.subplots(2, 2, figsize=(12,8))

sls = ['-', '--']

for i, (ax, unshielded, shielded) in enumerate([(ax0, None,sinks0), (ax1,sinks1, sinks4),
                                                (ax2,sinks2, sinks5), (ax3,sinks3, sinks6)]):
    if unshielded is not None:
        for j,sinkID in enumerate(unshielded.ID.unique()):
            growth = unshielded[unshielded.ID == sinkID]['mass']
            ax = growth.plot(ax=ax, grid=False, lw=3, color='.85')
    for j,sinkID in enumerate(shielded.ID.unique()):
        growth = shielded[shielded.ID == sinkID]['mass']
        ax = growth.plot(ax=ax, grid=False, lw=3)#, ls=sls[j/6])
    ax.text(.03,.93, simname[name[i]], ha='left', va='top', size=20, transform=ax.transAxes)

ax0.xaxis.set_visible(False)
ax1.xaxis.set_visible(False)
ax1.yaxis.set_visible(False)
ax3.yaxis.set_visible(False)


for ax in fig.axes:
    ax.set_xlim(-20,5000)
    ax.set_ylim(0,35)
ax0.set_yticks((5,10,15,20,25,30,35))
ax2.set_yticks((0,5,10,15,20,25,30))
ax2.set_xticks((0,1000,2000,3000,4000))
ax3.set_xticks((1000,2000,3000,4000,5000))

ax0.set_ylabel('Mass [$M_{\odot}$]')
ax2.set_ylabel('Mass [$M_{\odot}$]')
ax2.set_xlabel('Time [yr]')
ax3.set_xlabel('Time [yr]')

#ax0.text(.03,.97, r'J = 0', ha='left', va='top', size=25, transform=ax0.transAxes)
#ax1.text(.03,.97, r'J = J$_{0}$', ha='left', va='top', size=25, transform=ax1.transAxes)
#ax2.text(.03,.97, r'J = 10 J$_{0}$', ha='left', va='top', size=25, transform=ax2.transAxes)
#ax3.text(.03,.97, r'J = 100 J$_{0}$', ha='left', va='top', size=25, transform=ax3.transAxes)

plt.tight_layout(w_pad=0.0,h_pad=0.0)
fig.savefig('figures/sinks/sink_masses.pdf', bbox_inches='tight')
#mpl.rc('axes', color_cycle=['k','b','g','r','k','b','g','r'])

In [ ]:
total_mass = []
for i,data in enumerate([data0, data1, data2, data3]):
    mtot = data[['time','mass']].groupby('time').sum()
    mtot.rename(columns = {'mass':name[i]}, inplace=True)
    total_mass.append(mtot)

In [ ]:
slw = 3
with sns.color_palette(n_colors=4):
    fig, ax = plt.subplots()
df = total_mass[0]
ax.plot(df.index, df.vanilla, label=simname['vanilla'], lw=slw, color='.25')
for i,df in enumerate(total_mass[1:]):
    ax.plot(df.index, df[name[i+1]], label=simname[name[i+1]])
leg = plt.legend(loc=0)
plt.xlim(0,5000)
plt.ylim(0,80)
#plt.ylim(1,100); plt.semilogy()

plt.ylabel('Mass [$M_{\odot}$]')
plt.xlabel('Time [yr]')
plt.savefig('figures/sinks/total_sink_mass.png', bbox_inches='tight')

In [ ]:
for i in xrange(4):
    df = total_mass[i]
    print df[df.index >= 5000].head(1), '\n'

In [ ]:
most_massive_sink = []
for i,data in enumerate([data0, data1, data2, data3]):
    mms = data[['time','mass']].groupby('time').max()
    mms.rename(columns = {'mass':name[i]}, inplace=True)
    most_massive_sink.append(mms)

In [ ]:
slw = 3
count = 0
with sns.color_palette(n_colors=4):
    fig, ax = plt.subplots()
df = most_massive_sink[0]
ax.plot(df.index, df.vanilla, label=simname['vanilla'], lw=slw, color='.25')
for i,df in enumerate(most_massive_sink[1:]):
    ax.plot(df.index, df[name[i+1]], label=simname[name[i+1]])
  
leg = plt.legend(loc=0)
plt.xlim(0,5000)
plt.ylim(0,30)
#plt.ylim(1,100); plt.semilogy()

plt.ylabel('Mass [$M_{\odot}$]')
plt.xlabel('Time [yr]')
plt.title('Most Massive Sink')
plt.savefig('figures/sinks/most_massive_sink.png', bbox_inches='tight')